In [41]:
import os
import shutil

mount_point = '/content/drive'

# Check if the directory exists and is not empty
if os.path.isdir(mount_point) and os.listdir(mount_point):
    print(f"Clearing the mount point directory: {mount_point}")
    try:
        # Remove all contents of the directory
        for item in os.listdir(mount_point):
            item_path = os.path.join(mount_point, item)
            if os.path.isfile(item_path):
                os.remove(item_path)
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)
        print("Mount point directory cleared.")
    except Exception as e:
        print(f"Error clearing mount point directory: {e}")
else:
    print(f"Mount point directory {mount_point} is already empty or does not exist.")

# Attempt to mount Google Drive again
from google.colab import drive
try:
    drive.mount(mount_point)
    print("Google Drive mounted successfully.")
except ValueError as e:
    print(f"Error mounting Google Drive: {e}")

Clearing the mount point directory: /content/drive
Mount point directory cleared.
Mounted at /content/drive
Google Drive mounted successfully.


In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
import gc
gc.enable()

In [44]:
import os
import cv2
import numpy as np
import pathlib
import gc

base_classification_dir = pathlib.Path("/content/drive/MyDrive/pest_classifier/classification")

# Select only 5 categories for training
selected_category_names = [
    "rice leaf roller",
    "rice leaf caterpillar",
    "paddy stem maggot",
    "asiatic rice borer",
    "yellow rice borer"
]

directory_names = [str(i) for i in range(0, 102)]

train_dir = base_classification_dir / "train"
validation_dir = base_classification_dir / "val"
test_dir = base_classification_dir / "test"


print("\nDataset Paths Defined.")
print(f"Training data path: {train_dir}")
print(f"Validation data path: {validation_dir}")
print(f"Test data path: {test_dir}")


Dataset Paths Defined.
Training data path: /content/drive/MyDrive/pest_classifier/classification/train
Validation data path: /content/drive/MyDrive/pest_classifier/classification/val
Test data path: /content/drive/MyDrive/pest_classifier/classification/test


In [45]:
import psutil
print(f"Memory used: {psutil.virtual_memory().percent}%")


Memory used: 14.9%


In [46]:
import os

train_dir = "/content/drive/MyDrive/pest_classifier/classification/train"
folders = sorted(os.listdir(train_dir))
print("Folders found:", folders)
print("Total folders:", len(folders))


Folders found: ['0', '1', '10', '100', '101', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']
Total folders: 102


In [47]:
# Define image size and batch size
img_size = 224
batch_size = 32

print(f"Image size set to: {img_size}x{img_size}")
print(f"Batch size set to: {batch_size}")

Image size set to: 224x224
Batch size set to: 32


In [48]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, MobileNetV2, InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

## Define resnet model

### Subtask:
Build a ResNet model using pre-trained weights (e.g., ResNet50, ResNet101) and add custom layers for your classification task.


**Reasoning**:
Build the ResNet model according to the instructions.



In [49]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50

base_resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Freeze the base model layers
for layer in base_resnet_model.layers:
    layer.trainable = False

x = base_resnet_model.output

x = base_resnet_model.GlobalAveragePooling2D()(x)

x = layers.Dense(512, activation='relu')(x)

x = layers.Dropout(0.5)(x)

# Update the output layer to have the number of selected categories
predictions = layers.Dense(len(selected_category_names), activation='softmax')(x)

resnet_model = models.Model(inputs=base_resnet_model.input, outputs=predictions)

resnet_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast (Cast)         │ (None, 224, 224,  │          0 │ input_layer_3[0]… │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ cast[0][0]        │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c

 Total params: 24,639,365 (93.99 MB)

 Trainable params: 1,051,653 (4.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

## Define mobilenet model

### Subtask:
Build a MobileNet model using pre-trained weights (e.g., MobileNetV2) and add custom layers for your classification task.


**Reasoning**:
Build the MobileNetV2 model by loading pre-trained weights, freezing the base layers, and adding custom layers for classification, then print the model summary.



In [50]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

base_mobilenet_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

for layer in base_mobilenet_model.layers:
    layer.trainable = False


x = base_mobilenet_model.output

x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(512, activation='relu')(x)

x = layers.Dropout(0.5)(x)

# Update the output layer to have the number of selected categories
predictions = layers.Dense(len(selected_category_names), activation='softmax')(x)


mobilenet_model = models.Model(inputs=base_mobilenet_model.input, outputs=predictions)


mobilenet_model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast_1 (Cast)       │ (None, 224, 224,  │          0 │ input_layer_4[0]… │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ cast_1[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis

 Total params: 2,916,421 (11.13 MB)

 Trainable params: 658,437 (2.51 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

## Define inceptionv3 model

### Subtask:
Build an InceptionV3 model using pre-trained weights and add custom layers for your classification task.


**Reasoning**:
Build the InceptionV3 model by loading the pre-trained weights, adding custom layers, and printing the summary.



In [51]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define image size
img_size = 224

base_inceptionv3_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

for layer in base_inceptionv3_model.layers:
    layer.trainable = False

x = base_inceptionv3_model.output

x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(512, activation='relu')(x)

x = layers.Dropout(0.5)(x)

# Update the output layer to have the number of selected categories
predictions = layers.Dense(len(selected_category_names), activation='softmax')(x)

inceptionv3_model = models.Model(inputs=base_inceptionv3_model.input, outputs=predictions)

inceptionv3_model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast_2 (Cast)       │ (None, 224, 224,  │          0 │ input_layer_5[0]… │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_94 (Conv2D)  │ (None, 111, 111,  │        864 │ cast_2[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 111, 111,  │         96 │ conv2d_94[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_94       │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_95 (Conv2D)  │ (None, 109, 109,  │      9,216 │ activation_94[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_95[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_95       │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_96 (Conv2D)  │ (None, 109, 109,  │     18,432 │ activation_95[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_96[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_96       │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 54, 54,    │          0 │ activation_96[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_97 (Conv2D)  │ (None, 54, 54,    │      5,120 │ max_pooling2d_6[… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_97[0][0]   │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_97       │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_98 (Conv2D)  │ (None, 52, 52,    │    138,240 │ activation_97[0]… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_98[0][0] 

 Total params: 22,854,437 (87.18 MB)

 Trainable params: 1,051,653 (4.01 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [69]:
from sklearn.utils import class_weight
import numpy as np


train_counts = {'rice leaf roller': 669, 'rice leaf caterpillar': 292, 'paddy stem maggot': 156, 'asiatic rice borer': 631, 'yellow rice borer': 302}
selected_category_names = [
    "rice leaf roller",
    "rice leaf caterpillar",
    "paddy stem maggot",
    "asiatic rice borer",
    "yellow rice borer"
]
# Create a list of all labels based on the counts
all_labels = []
for i, category_name in enumerate(selected_category_names):
    all_labels.extend([i] * train_counts[category_name])

# Calculate class weights
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(all_labels),
    y=all_labels
)

# Convert to a dictionary
class_weight_dict = dict(enumerate(class_weights))

print("Calculated class weights:")
print(class_weight_dict)


resnet_model.compile(optimizer=Adam(learning_rate=0.001),
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

mobilenet_model.compile(optimizer=Adam(learning_rate=0.001),
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

inceptionv3_model.compile(optimizer=Adam(learning_rate=0.001),
                          loss='sparse_categorical_crossentropy',
                          metrics=['accuracy'])

print("Models compiled successfully.")

Calculated class weights:
{0: np.float64(0.6128550074738416), 1: np.float64(1.404109589041096), 2: np.float64(2.628205128205128), 3: np.float64(0.6497622820919176), 4: np.float64(1.3576158940397351)}
Models compiled successfully.


In [53]:

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
print("Mixed precision enabled.")

Mixed precision enabled.


In [54]:
import tensorflow as tf
import os

def load_and_preprocess_image(image_path, label):
    """Loads and preprocesses an image."""
    # Ensure image_path is treated as a string
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [img_size, img_size])
    img = img / 255.0
    return img, label

def create_dataset(directory, category_names, shuffle=True):
    """Creates a tf.data.Dataset from image files in a directory."""
    image_paths = []
    labels = []
    # Define the mapping from category name to original index based on the full list
    full_directory_names = [str(i) for i in range(0, 102)]
    category_to_original_index = {name: full_directory_names.index(str(i)) for i, name in enumerate(selected_category_names)}


    for i, category in enumerate(category_names):
        # Map the selected category name back to its original index
        if category in category_to_original_index:
            original_index = category_to_original_index[category]
            category_dir = os.path.join(directory, str(original_index))
            if os.path.exists(category_dir):
                for img_name in os.listdir(category_dir):
                    # Encode the image path to bytes
                    image_paths.append(os.path.join(category_dir, img_name).encode('utf-8'))
                    labels.append(i) # Use the new index based on the selected categories

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)

    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.batch(batch_size)

    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

    return dataset

train_dir = "/content/drive/MyDrive/pest_classifier/classification/train"
validation_dir = "/content/drive/MyDrive/pest_classifier/classification/val"
test_dir = "/content/drive/MyDrive/pest_classifier/classification/test"

# Use the selected category names to create the datasets
train_dataset = create_dataset(train_dir, selected_category_names)
val_dataset = create_dataset(validation_dir, selected_category_names, shuffle=False)
test_dataset = create_dataset(test_dir, selected_category_names, shuffle=False)

print("tf.data.Dataset created for train, validation, and test sets with selected categories.")

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

resnet_checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/MyDrive/best_resnet_model.weights.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

mobilenet_checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/MyDrive/best_mobilenet_model.weights.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

inceptionv3_checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/MyDrive/best_inceptionv3_model.weights.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

early_stopping_callback = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    mode='max',
    restore_best_weights=True,
    verbose=1
)

tf.data.Dataset created for train, validation, and test sets with selected categories.


In [55]:
print("\nTraining MobileNetV2 model...")
mobilenet_history = mobilenet_model.fit(
    train_dataset,
    epochs=5,
    validation_data=val_dataset,
    callbacks=[mobilenet_checkpoint_callback, early_stopping_callback]
)

tf.keras.backend.clear_session()
gc.collect()



Training MobileNetV2 model...
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.4704 - loss: 2.5166
Epoch 1: val_accuracy improved from -inf to 0.52493, saving model to /content/drive/MyDrive/best_mobilenet_model.weights.h5


65/65 ━━━━━━━━━━━━━━━━━━━━ 394s 6s/step - accuracy: 0.4701 - loss: 2.5086 - val_accuracy: 0.5249 - val_loss: 1.2006
Restoring model weights from the end of the best epoch: 1.


0

In [70]:
gc.collect()

print("Training ResNet model...")
resnet_history = resnet_model.fit(
    train_dataset,
    epochs=3,
    validation_data=val_dataset,
    callbacks=[resnet_checkpoint_callback, early_stopping_callback]
)


tf.keras.backend.clear_session()
gc.collect()


Training ResNet model...
Epoch 1/3
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step - accuracy: 0.4560 - loss: 1.4627
Epoch 1: val_accuracy did not improve from 0.31085
65/65 ━━━━━━━━━━━━━━━━━━━━ 57s 695ms/step - accuracy: 0.4546 - loss: 1.4632 - val_accuracy: 0.3109 - val_loss: 1.5227
Epoch 2/3
64/65 ━━━━━━━━━━━━━━━━━━━━ 0s 492ms/step - accuracy: 0.3096 - loss: 1.5116
Epoch 2: val_accuracy did not improve from 0.31085
65/65 ━━━━━━━━━━━━━━━━━━━━ 65s 505ms/step - accuracy: 0.3102 - loss: 1.5121 - val_accuracy: 0.3109 - val_loss: 1.5008
Epoch 3/3
64/65 ━━━━━━━━━━━━━━━━━━━━ 0s 504ms/step - accuracy: 0.2603 - loss: 1.5157
Epoch 3: val_accuracy did not improve from 0.31085
65/65 ━━━━━━━━━━━━━━━━━━━━ 34s 518ms/step - accuracy: 0.2615 - loss: 1.5157 - val_accuracy: 0.3109 - val_loss: 1.4895
Restoring model weights from the end of the best epoch: 1.


0

In [71]:

print("\nTraining InceptionV3 model...")
inceptionv3_history = inceptionv3_model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    callbacks=[inceptionv3_checkpoint_callback, early_stopping_callback]
)


Training InceptionV3 model...
Epoch 1/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step - accuracy: 0.3051 - loss: 1.8995
Epoch 1: val_accuracy did not improve from 0.62170
65/65 ━━━━━━━━━━━━━━━━━━━━ 59s 670ms/step - accuracy: 0.3050 - loss: 1.8971 - val_accuracy: 0.3871 - val_loss: 1.3609
Epoch 2/10
64/65 ━━━━━━━━━━━━━━━━━━━━ 0s 429ms/step - accuracy: 0.3551 - loss: 1.5537
Epoch 2: val_accuracy did not improve from 0.62170
65/65 ━━━━━━━━━━━━━━━━━━━━ 60s 444ms/step - accuracy: 0.3541 - loss: 1.5536 - val_accuracy: 0.4194 - val_loss: 1.4170
Epoch 3/10
64/65 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step - accuracy: 0.2360 - loss: 1.5373
Epoch 3: val_accuracy did not improve from 0.62170
65/65 ━━━━━━━━━━━━━━━━━━━━ 45s 511ms/step - accuracy: 0.2377 - loss: 1.5373 - val_accuracy: 0.4223 - val_loss: 1.4190
Epoch 4/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 459ms/step - accuracy: 0.2716 - loss: 1.5539
Epoch 4: val_accuracy did not improve from 0.62170
65/65 ━━━━━━━━━━━━━━━━━━━━ 39s 480ms/step - accuracy: 0.2722 - loss

In [72]:
inceptionv3_train_results = inceptionv3_model.evaluate(train_dataset)

65/65 ━━━━━━━━━━━━━━━━━━━━ 31s 480ms/step - accuracy: 0.2236 - loss: 1.5139


In [73]:
inceptionv3_test_results = inceptionv3_model.evaluate(test_dataset)

print(f"InceptionV3 Test Loss: {inceptionv3_test_results[0]:.4f}")
print(f"InceptionV3 Test Accuracy: {inceptionv3_test_results[1]:.4f}")

33/33 ━━━━━━━━━━━━━━━━━━━━ 7s 203ms/step - accuracy: 0.5713 - loss: 1.2286
InceptionV3 Test Loss: 1.3750
InceptionV3 Test Accuracy: 0.5121


In [1]:

print("\nEvaluating ResNet model on the test dataset...")
resnet_test_results = resnet_model.evaluate(test_dataset)

print(f"ResNet Test Loss: {resnet_test_results[0]:.4f}")
print(f"ResNet Test Accuracy: {resnet_test_results[1]:.4f}")


Evaluating ResNet model on the test dataset...


NameError: name 'resnet_model' is not defined

In [74]:


print("\nEvaluating MobileNetV2 model on the test dataset...")
mobilenet_test_results = mobilenet_model.evaluate(test_dataset)

print(f"MobileNetV2 Test Loss: {mobilenet_test_results[0]:.4f}")
print(f"MobileNetV2 Test Accuracy: {mobilenet_test_results[1]:.4f}")

print("\nEvaluating InceptionV3 model on the test dataset...")



Evaluating ResNet model on the test dataset...
33/33 ━━━━━━━━━━━━━━━━━━━━ 7s 212ms/step - accuracy: 0.1277 - loss: 1.5916
ResNet Test Loss: 1.5237
ResNet Test Accuracy: 0.3071

Evaluating MobileNetV2 model on the test dataset...
33/33 ━━━━━━━━━━━━━━━━━━━━ 12s 230ms/step - accuracy: 0.6626 - loss: 0.9936
MobileNetV2 Test Loss: 1.2008
MobileNetV2 Test Accuracy: 0.5442

Evaluating InceptionV3 model on the test dataset...


The error `ValueError: Mountpoint must not already contain files` indicates that the directory you are trying to mount Google Drive to (`/content/drive`) is not empty. Google Drive requires an empty directory to mount.

To fix this, you can either:

1.  **Clear the directory**: If the files in `/content/drive` are not important, you can delete them. However, be cautious when deleting files from mounted directories.
2.  **Unmount and Remount**: If the drive was previously mounted incorrectly, you can try unmounting and then remounting it.

Since cell `4bd3ef52` indicates that the zip file was not found, it's likely that the drive was not mounted successfully in the first place. Let's try unmounting and then remounting Google Drive with the code below:

In [76]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, RandomBrightness

# Define the data augmentation pipeline
data_augmentation = Sequential([
    RandomFlip("horizontal_and_vertical"),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomBrightness(factor=0.2),
], name="data_augmentation")

def load_and_preprocess_image(image_path, label, augment=False):
    """Loads and preprocesses an image, optionally applying augmentation."""
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [img_size, img_size])
    img = img / 255.0 # Normalize to [0, 1]
    if augment:
        img = data_augmentation(img, training=True) # Apply augmentation during training
    return img, label

def create_dataset(directory, category_names, shuffle=True, augment=False):
    """Creates a tf.data.Dataset from image files in a directory."""
    image_paths = []
    labels = []
    full_directory_names = [str(i) for i in range(0, 102)]
    category_to_original_index = {name: full_directory_names.index(str(i)) for i, name in enumerate(selected_category_names)}

    for i, category in enumerate(category_names):
        if category in category_to_original_index:
            original_index = category_to_original_index[category]
            category_dir = os.path.join(directory, str(original_index))
            if os.path.exists(category_dir):
                for img_name in os.listdir(category_dir):
                    image_paths.append(os.path.join(category_dir, img_name).encode('utf-8'))
                    labels.append(i)

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)

    # Use lambda to pass the 'augment' parameter to load_and_preprocess_image
    dataset = dataset.map(lambda x, y: load_and_preprocess_image(x, y, augment=augment), num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.batch(batch_size)

    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

    return dataset

# Recreate the train_dataset with augmentation
train_dataset = create_dataset(train_dir, selected_category_names, augment=True)

# Recreate validation and test datasets without augmentation
val_dataset = create_dataset(validation_dir, selected_category_names, shuffle=False, augment=False)
test_dataset = create_dataset(test_dir, selected_category_names, shuffle=False, augment=False)

print("Data augmentation pipeline defined.")
print("tf.data.Dataset created/recreated with augmentation for training set.")
print("Validation and test datasets created/recreated without augmentation.")

Data augmentation pipeline defined.
tf.data.Dataset created/recreated with augmentation for training set.
Validation and test datasets created/recreated without augmentation.


In [77]:
import gc
import tensorflow as tf

gc.collect()

print("\nTraining ResNet model...")
resnet_history = resnet_model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    callbacks=[resnet_checkpoint_callback, early_stopping_callback]
)

tf.keras.backend.clear_session()
gc.collect()

print("\nTraining MobileNetV2 model...")
mobilenet_history = mobilenet_model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    callbacks=[mobilenet_checkpoint_callback, early_stopping_callback]
)

tf.keras.backend.clear_session()
gc.collect()

print("\nTraining InceptionV3 model...")
inceptionv3_history = inceptionv3_model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    callbacks=[inceptionv3_checkpoint_callback, early_stopping_callback]
)


Training ResNet model...
Epoch 1/10
64/65 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step - accuracy: 0.2889 - loss: 1.5252
Epoch 1: val_accuracy did not improve from 0.31085
65/65 ━━━━━━━━━━━━━━━━━━━━ 34s 496ms/step - accuracy: 0.2895 - loss: 1.5251 - val_accuracy: 0.3109 - val_loss: 1.4975
Epoch 2/10
64/65 ━━━━━━━━━━━━━━━━━━━━ 0s 488ms/step - accuracy: 0.2484 - loss: 1.5044
Epoch 2: val_accuracy did not improve from 0.31085
65/65 ━━━━━━━━━━━━━━━━━━━━ 41s 501ms/step - accuracy: 0.2499 - loss: 1.5047 - val_accuracy: 0.3109 - val_loss: 1.5199
Epoch 3/10
64/65 ━━━━━━━━━━━━━━━━━━━━ 0s 492ms/step - accuracy: 0.2314 - loss: 1.5156
Epoch 3: val_accuracy did not improve from 0.31085
65/65 ━━━━━━━━━━━━━━━━━━━━ 42s 525ms/step - accuracy: 0.2333 - loss: 1.5156 - val_accuracy: 0.3109 - val_loss: 1.4995
Epoch 4/10
64/65 ━━━━━━━━━━━━━━━━━━━━ 0s 609ms/step - accuracy: 0.2429 - loss: 1.5274
Epoch 4: val_accuracy did not improve from 0.31085
65/65 ━━━━━━━━━━━━━━━━━━━━ 48s 640ms/step - accuracy: 0.2446 - loss: 1.5

KeyboardInterrupt: 

**Reasoning**:
The error `ValueError: Argument output must have rank (ndim) target.ndim - 1. Received: target.shape=(None,), output.shape=(None, 3, 3, 5)` indicates that the output shape of the ResNet model is not compatible with the sparse categorical crossentropy loss function. This is likely due to the MaxPooling2D layer applied after the base model output, which results in a 4D tensor output instead of a 2D tensor suitable for the dense layer. The model architecture needs to be adjusted to flatten or pool the output before the final dense layer.



In [67]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import gc

# Rebuild the ResNet model with GlobalAveragePooling2D
base_resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Freeze the base model layers
for layer in base_resnet_model.layers:
    layer.trainable = False

x = base_resnet_model.output

# Replace MaxPooling2D with GlobalAveragePooling2D
x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(512, activation='relu')(x)

x = layers.Dropout(0.5)(x)

# Update the output layer to have the number of selected categories
predictions = layers.Dense(len(selected_category_names), activation='softmax')(x)

resnet_model = models.Model(inputs=base_resnet_model.input, outputs=predictions)

resnet_model.compile(optimizer=Adam(learning_rate=0.001),
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

print("\nRebuilt and compiled ResNet model with GlobalAveragePooling2D.")
resnet_model.summary()

# Now, retrain the models
gc.collect()

print("\nTraining ResNet model...")
resnet_history = resnet_model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    callbacks=[resnet_checkpoint_callback, early_stopping_callback]
)

tf.keras.backend.clear_session()
gc.collect()

print("\nTraining MobileNetV2 model...")
mobilenet_history = mobilenet_model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    callbacks=[mobilenet_checkpoint_callback, early_stopping_callback]
)

tf.keras.backend.clear_session()
gc.collect()

print("\nTraining InceptionV3 model...")
inceptionv3_history = inceptionv3_model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    callbacks=[inceptionv3_checkpoint_callback, early_stopping_callback]
)


Rebuilt and compiled ResNet model with GlobalAveragePooling2D.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,639,365 (93.99 MB)

 Trainable params: 1,051,653 (4.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)


Training ResNet model...
Epoch 1/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 530ms/step - accuracy: 0.3791 - loss: 1.6897
Epoch 1: val_accuracy improved from -inf to 0.15249, saving model to /content/drive/MyDrive/best_resnet_model.weights.h5


65/65 ━━━━━━━━━━━━━━━━━━━━ 64s 737ms/step - accuracy: 0.3784 - loss: 1.6892 - val_accuracy: 0.1525 - val_loss: 1.5619
Epoch 2/10
64/65 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step - accuracy: 0.3794 - loss: 1.5556
Epoch 2: val_accuracy improved from 0.15249 to 0.31085, saving model to /content/drive/MyDrive/best_resnet_model.weights.h5


65/65 ━━━━━━━━━━━━━━━━━━━━ 37s 567ms/step - accuracy: 0.3792 - loss: 1.5554 - val_accuracy: 0.3109 - val_loss: 1.5212
Epoch 3/10
64/65 ━━━━━━━━━━━━━━━━━━━━ 0s 512ms/step - accuracy: 0.2338 - loss: 1.5768
Epoch 3: val_accuracy did not improve from 0.31085
65/65 ━━━━━━━━━━━━━━━━━━━━ 40s 546ms/step - accuracy: 0.2359 - loss: 1.5756 - val_accuracy: 0.3109 - val_loss: 1.4957
Epoch 4/10
14/65 ━━━━━━━━━━━━━━━━━━━━ 28s 560ms/step - accuracy: 0.2173 - loss: 1.5656

KeyboardInterrupt: 